In [ ]:
from datetime import datetime, timedelta
import os
import io
from google.colab import files
import gspread
!pip install creds
# from creds import client_secret
import creds
import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import date
from google.colab import drive
!pip install import-ipynb
import import_ipynb
# from helper_functions import *
from shapely.geometry import Point, LineString
import pickle
from collections import Counter



final map file for egt
gogpt and goget we can use the dd rob filtered and prepared
ggit use the json files Baird filtered and prepared



In [ ]:
egt_dd = '/content/drive/Shareddrives/GEM Shared Drive/Programs/Special and Multi-Program Projects/Data team/Tracker official releases (data team copies)/Europe-Gas-Tracker-2025-01-30 DATA TEAM COPY.xlsx'
# tabs
# gas plants = H2 Proposals at Oil & Gas Plant
# gas extraction = Oil & Gas Extraction
egt_terminals = '/content/drive/Shareddrives/GEM Shared Drive/Programs/Special and Multi-Program Projects/Data team/Tracker official releases (data team copies)/GEM-EGT-Terminals-2025-02 DATA TEAM COPY.geojson'

egt_pipelines = '/content/drive/Shareddrives/GEM Shared Drive/Programs/Special and Multi-Program Projects/Data team/Tracker official releases (data team copies)/GEM-EGT-Gas-Hydrogen-Pipelines-2025-02 DATA TEAM COPY.geojson'

In [ ]:
# read in conversion file
conversion = '/content/drive/Shareddrives/GEM Shared Drive/Programs/Special and Multi-Program Projects/Data team/GEM tracker maps (data team)/GEM maps - conversion of values to TJ_y for scaling - data.xlsx'

conversion_df = pd.read_excel(conversion, sheet_name='data')

conversion_df = conversion_df[['tracker', 'type', 'original units', 'conversion factor (capacity/production to common energy equivalents, TJ/y)']]
conversion_df = conversion_df.rename(columns={'conversion factor (capacity/production to common energy equivalents, TJ/y)': 'conversion_factor', 'original units': 'original_units'})
# conversion_df['tracker'] = conversion_df['tracker'].apply(lambda x: x.strip())
print(conversion_df.head())



  tracker             type original_units  conversion_factor
0    GCPT       coal power             MW                 55
1   GOGPT    oil/gas power             MW                 26
2    GBPT  bioenergy power             MW                 59
3    GNPT          nuclear             MW                 59
4   GSPT     solar thermal             MW                 10


In [ ]:
term = gpd.read_file(egt_terminals)
term['tracker'] = 'term'
pipes = gpd.read_file(egt_pipelines)
pipes['tracker'] = 'pipes'
gogpt_hy = pd.read_excel(egt_dd, sheet_name='H2 Proposals at Oil & Gas Plant')
gogpt_hy['tracker'] = 'plants_hy'

gogpt = pd.read_excel(egt_dd, sheet_name='Oil & Gas Plants')
gogpt['tracker'] = 'plants'

goget = pd.read_excel(egt_dd, sheet_name='Oil & Gas Extraction')
goget['tracker'] = 'extraction'

# convert term and pipes to gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

pipes = gpd.GeoDataFrame(pipes, geometry='geometry', crs="EPSG:4326")
term = gpd.GeoDataFrame(term, geometry='geometry', crs="EPSG:4326")

list_to_be_merged = [term, pipes, gogpt, goget, gogpt_hy]

In [ ]:
# find out if all duplicates between gogpt and gogpt_hy are eliminated by the In Oil & Gas Plants tab? tab
# this is necessary because concatting all trackers with verify_integrity =True is throwing errors
# full_list_of_urls = pd.concat([gogpt, gogpt_hy], verify_integrity=True, ignore_index=True)
# full_list_of_urls = full_list_of_urls['Wiki URL']
# full_list_of_urls # 2194
# unique_urls = full_list_of_urls.unique()
# print(len(unique_urls)) # 1166

# full_list_of_ids = pd.concat([gogpt, gogpt_hy], verify_integrity=True, ignore_index=True)
# full_list_of_ids = full_list_of_ids['GEM location ID']
# full_list_of_ids # 2194
# unique_ids = full_list_of_ids.unique()
# print(len(unique_ids)) # 1166

In [ ]:
def convert_coords_to_point(df):
    crs = 'EPSG: 4326'
    geometry_col = 'geometry'
    df = df.reset_index(drop=True)
    for row in df.index:
        df.loc[row,'geometry'] = Point(df.loc[row,'longitude'], df.loc[row,'latitude'])
    gdf = gpd.GeoDataFrame(df, geometry=geometry_col, crs=crs)

    return gdf

In [ ]:
def adjusting_geometry(df):


    # convert to gdf
    tracker = df['tracker'].iloc[0]
    # print(tracker)
    # input('check')
    df.columns = df.columns.str.lower()

    if 'geometry' not in df.columns:
        if 'latitude' in df.columns and 'longitude' in df.columns:

            df = convert_coords_to_point(df)
            crs = 'EPSG: 4326'
            geometry_col = 'geometry'
            df = gpd.GeoDataFrame(df, geometry=geometry_col, crs=crs)
        else:
            print(df.columns)
            input('no lat long or geo?')


    else:
      # if geometry is null or empty then use lat long to conver to a point

        df['geometry'] = df.apply(lambda x: Point(x['longitude'], x['latitude']) if x['geometry'] is None or x['geometry'] == '' or x['geometry'] == 'null' else x['geometry'], axis=1)
        crs = 'EPSG: 4326'
        geometry_col = 'geometry'
        df = gpd.GeoDataFrame(df, geometry=geometry_col, crs=crs)
        print(len(df))
        df = df.dropna(subset='geometry')
        # print(len(df))
        # input('check change in length after dropping na')

    return df

gdf_to_be_merged = []
for df in list_to_be_merged:
    df = adjusting_geometry(df)
    print(df['geometry'])
    # how can I make sure this updated gdf is now
    gdf_to_be_merged.append(df)





158
0      POINT (19.48333 40.68333)
1         POINT (3.22241 51.353)
2         POINT (3.22241 51.353)
3         POINT (3.22241 51.353)
4         POINT (3.22241 51.353)
                 ...            
153    POINT (-3.67666 56.02248)
154     POINT (-8.2328 51.82681)
155    POINT (13.59835 54.47838)
156    POINT (13.59835 54.47838)
157    POINT (-1.23496 54.57423)
Name: geometry, Length: 158, dtype: geometry
1272
0       LINESTRING (-7.88456 43.18086, -7.85297 43.44965)
1       LINESTRING (-7.85297 43.44965, -8.25672 43.46061)
2       LINESTRING (-8.15784 43.44666, -8.23493 43.267...
3       MULTILINESTRING ((7.5963 47.8351, 7.5892 47.79...
4       LINESTRING (13.98741 42.01967, 14.00469 41.960...
                              ...                        
1267                             GEOMETRYCOLLECTION EMPTY
1268                             GEOMETRYCOLLECTION EMPTY
1269                             GEOMETRYCOLLECTION EMPTY
1270                             GEOMETRYCOLLECTION EMPTY
127

In [ ]:
finalcols = ['status-legend','status', 'fuel-filter', 'tracker-custom', 'maturity', 'pci-list', 'scaling-capacity', 'name', 'areas',
        'unit-name', 'owner', 'parent', 'capacity-table', 'units-of-m','start-year', 'prod-gas', 'prod-year-gas', 'tracker-display',
        'areas-subnat-sat-display','other-name', 'local-name','capacity', 'capacity-details']

In [ ]:
# fuel function options: 'hy', 'blend', 'methane'

# For goget fuel_filter == methane

# For power plants, if data in the fuel column includes the text "hydrogen" it can be included, and then the column "H2 usage proposed %" gives the percent blend. The projects can be assigned based on the numbers in that column: 0-100 = blended, not found = blended, or 100 = 100%

# For terminals, if fuel = anything other than LNG or oil, it's a hydrogen derivative terminal and can be labeled 100% (Julie and Baird, I'm not sure this system makes total sense when it comes to hydrogen terminals, but oh well for now)

# For pipelines, if fuel = gas, it can be labeled "None" on the map. If fuel = hydrogen, see the column H2%. the H2% column has some text in it, as a warning (e.g., 10-100%, "up to 100%", etc.) — my advice for the logic is:
# (1) if it's === 100.00% or blank, treat it as 100% hydrogen
# (2) if it's anything else (10%, 90%, 10-90%, "up to 100%"), treat it as blended

In [ ]:
def fuel_filter(df):
  df.columns = df.columns.str.lower()
  df.columns = df.columns.str.replace(' ', '-')
  df['fuel-filter'] = 'methane'
  if df['tracker'].iloc[0] == 'extraction':
      df['fuel-filter'] = 'methane'

  elif df['tracker'].iloc[0] == 'plants_hy':
    # if hydrogen in the fuel column then if H2 usage proposed % to see if it's blend or 100% hydrogen
    # df['fuel-filter'] = np.where(df['fuel'].str.lower().str.contains('hydrogen'), df['h2-usage-proposed-%'], 'methane')
    # find all fuels that have hydrogen and [100] in h2-usage-proposed-%
    for row in df.index:
      # if hydrogen in fuel column
      if 'hydrogen' in df.loc[row, 'fuel'].lower():
        # print(df.loc[row, 'h2-usage-proposed-%'])
        if df.loc[row, 'h2-usage-proposed-%'] == 100:
          df.loc[row, 'fuel-filter'] = 'hy'
        else:
          df.loc[row, 'fuel-filter'] = 'blend'

  elif df['tracker'].iloc[0] == 'plants':
    df['fuel-filter'] = 'methane'

  elif df['tracker'].iloc[0] == 'term':
    df['fuel'] = df['fuel'].str.lower()
    df['fuel-filter'] = np.where((df['fuel'] != 'lng') & (df['fuel'] != 'oil'), 'hy', df['fuel-filter'])

  elif df['tracker'].iloc[0] == 'pipes':
    df['h2%'].fillna('', inplace=True)

    for row in df.index:
      if df.loc[row, 'fuel'].lower().strip() == 'hydrogen':
        # print(df.loc[row, 'h2%']) # h2 does not exist in json or dd so making them all blend
        # convert the column to a string after filling na
        df.loc[row, 'h2%'] = str(df.loc[row, 'h2%'])
        # print(df.loc[row, 'h2%']) # h2 does not exist in json or dd so making them all blend
        if df.loc[row, 'h2%'] == '100.00%':
          df.loc[row, 'fuel-filter'] = 'hy'
        elif df.loc[row, 'h2%'] == '':
          df.loc[row, 'fuel-filter'] = 'hy'
        else:
          df.loc[row, 'fuel-filter'] = 'blend'

  return df

for df in gdf_to_be_merged:
  df = fuel_filter(df)
  print(df['tracker'].iloc[0])
  print(set(df['fuel-filter'].to_list()))
  # show methane pipes that have ['pci6'] or pci 5

  if df['tracker'].iloc[0] == 'pipes':

    test = df[df['fuel-filter']=='methane']
    test.fillna('',inplace=True)
    test1 = test[test['pci5'] != '']
    test2 = test[test['pci6'] != '']
    test = pd.concat([test1, test2])
    print(test[['pci5', 'pci6', 'fuel-filter', 'tracker']])
    print(len(test))

term
{'hy', 'methane'}


<ipython-input-116-c505ff0878e5>:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['h2%'].fillna('', inplace=True)


pipes
{'blend', 'hy', 'methane'}
     pci5 pci6 fuel-filter tracker
39    yes          methane   pipes
40    yes          methane   pipes
41    yes          methane   pipes
42    yes          methane   pipes
134   yes  yes     methane   pipes
154   yes          methane   pipes
162   yes          methane   pipes
174   yes  yes     methane   pipes
188   yes          methane   pipes
213   yes  yes     methane   pipes
218   yes          methane   pipes
221   yes          methane   pipes
223   yes          methane   pipes
273   yes          methane   pipes
282   yes          methane   pipes
300   yes          methane   pipes
301   yes          methane   pipes
318   yes          methane   pipes
328   yes          methane   pipes
626   yes          methane   pipes
632   yes  yes     methane   pipes
675   yes          methane   pipes
1092  yes  yes     methane   pipes
1093  yes  yes     methane   pipes
1094  yes  yes     methane   pipes
1095  yes  yes     methane   pipes
1096  yes  yes     met

<ipython-input-116-c505ff0878e5>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.fillna('',inplace=True)


In [ ]:
# maturity function options: 'y', 'n', 'none'

# For goget maturity == n/a

# For power plants A hydrogen power project is "yes" if Status = Construction,
# MOU for H2 supply? = Y, Contract for H2 supply? = Y, Financing for supply of H2? = Y, or Co-located with electrolyzer/H2 production facility? = Y ND fuel is NOT methane only

# For terminals A hydrogen derivative terminal is "yes" on progress demonstrated if, in the data sheet, Status = Construction, FIDStatus = FID, AltFuelPrelimAgreement = yes, or AltFuelCallMarketInterest = yes AND fuel is NOT methane only

# For pipelines A hydrogen pipeline is "yes" if Status = Construction, PCI5 = yes, or PCI6 = yes AND fuel is NOT methane only



In [ ]:
def maturity(df):
  df['maturity'] = 'none' # starts as none

  for row in df.index:
      if df.loc[row, 'fuel-filter'] == 'methane':
          df.loc[row, 'maturity'] = 'none'
      else:
        if df.loc[row, 'tracker'] == 'plants_hy':
            df['maturity'] = np.where((df['status'] == 'Construction') | (df['mou-for-h2-supply?'] == 'Y') | (df['contract-for-h2-supply?'] == 'Y') | (df['financing-for-supply-of-h2?'] == 'Y') | (df['co-located-with-electrolyzer/h2-production-facility?'] == 'Y'), 'y','n')

        elif df.loc[row, 'tracker'] == 'term':
            # if df.loc[row, 'fuel-filter'] == 'methane':
            #     df.loc[row, 'maturity'] = 'none'
            # else:
              df['maturity'] = np.where((df['status'] == 'Construction') | (df['fidstatus'] == 'FID') | (df['altfuelprelimagreement'] == 'yes') | (df['altfuelcallmarketinterest'] == 'yes'), 'y','n')

        elif df.loc[row, 'tracker'] == 'pipes':
            # if df.loc[row, 'fuel-filter'] != 'methane':
                # df.loc[row, 'maturity'] = 'none'
                # break out of this for loop and go to the next row
                df['maturity'] = np.where((df['status'] == 'Construction') | (df['pci5'] == 'yes') | (df['pci6'] == 'yes'), 'y','n')

  # override any where fuel is methane
  for row in df.index:
      if df.loc[row, 'fuel-filter'] == 'methane':
          df.loc[row, 'maturity'] = 'none'

  return df

for df in gdf_to_be_merged:
    df = maturity(df)
    print(df['tracker'].iloc[0])
    print(set(df['maturity'].to_list()))
    # count of maturity equal none by tracker
    print(df[df['maturity']=='none'][['maturity', 'tracker']].groupby('tracker').count())
    # count of maturity equal y by tracker
    # print(df[['maturity', 'tracker']].groupby('tracker').count())

# term          149
# pipes
# {'none', 'n', 'y'}
#          maturity
# tracker
# pipes         928
# plants
# {'none'}
#          maturity
# tracker
# plants       2084
# extraction
# {'none'}
#             maturity
# tracker
# extraction       927
# plants_hy
# {'n', 'y'}

# term
# {'none', 'n', 'y'}
#          maturity
# tracker
# term           17
# pipes
# {'n', 'y'}
# Empty DataFrame
# Columns: [maturity]
# Index: []
# plants
# {'none'}
#          maturity
# tracker
# plants       2084
# extraction
# {'none'}
#             maturity
# tracker
# extraction       927
# plants_hy
# {'n', 'y'}
# Empty DataFrame
# Columns: [maturity]
# Index: []


term
{'y', 'none', 'n'}
         maturity
tracker          
term          149
pipes
{'y', 'none', 'n'}
         maturity
tracker          
pipes         928
plants
{'none'}
         maturity
tracker          
plants       2084
extraction
{'none'}
            maturity
tracker             
extraction       927
plants_hy
{'y', 'n'}
Empty DataFrame
Columns: [maturity]
Index: []


In [ ]:
# def merge_gogpts(gogpt, gogpt_hy):

#     # merge together gogpts

#     gogpts = [gogpt, gogpt_hy]

#     # remove all rows from gogpt_hy if column In Oil & Gas Plants tab? yes
#     gogpt_hy = gogpt_hy[gogpt_hy['in oil & gas plants tab?'] == 'no']
#     # merge gogpts and use plants as the tracker
#     gogpt = pd.concat(gogpts, sort=False, verify_integrity=True, ignore_index=True)
#     # if 'geometry' not in gogpt.columns:
#     gogpt = convert_coords_to_point(gogpt)

#     na_rows_gogpt = gogpt[gogpt['geometry'].isna()]
#     # print(len(na_rows_gogpt))
#     # na_rows.to_csv(f'/content/drive/MyDrive/output_from_colab/na-rows_gogpt.csv')
#     # input('check length of na rows for gogpt')
#     gogpt['tracker'] = 'plants'
#     if 'fuel-filter' not in gogpt.columns:
#         input('issue here')
#     return gogpt


# gogpt = merge_gogpts(gogpt, gogpt_hy)


In [ ]:
# gdf_to_be_merged.append(gogpt)


In [ ]:
def split_goget_ggit(df):

    if df['tracker'].iloc[0] == 'extraction':
        # df_goget_missing_units.to_csv('compilation_output/missing_gas_oil_unit_goget.csv')
        # gdf['tracker_custom'] = gdf.apply(lambda row: 'GOGET - gas' if row['Production - Gas (Million m³/y)'] != '' else 'GOGET-oil', axis=1)
        df['tracker_custom'] = 'GOGET-oil'

    elif df['tracker'].iloc[0] == 'term':
        # gdf_ggit_missing_units = df[df['facilitytype']=='']
        # gdf_ggit_missing_units.to_csv('/content/drive/MyDrive/output_from_colab/missing_gas_oil_unit_ggit.csv')
        # df = df[df['facilitytype']!='']
        df['tracker_custom'] = df.apply(lambda row: 'GGIT-import' if row['facilitytype'] == 'Import' else 'GGIT-export', axis=1)

    elif df['tracker'].iloc[0] == 'plants':
        df['tracker_custom'] = 'GOGPT'

    elif df['tracker'].iloc[0] == 'pipes':
        df['tracker_custom'] = 'GGIT'
    elif df['tracker'].iloc[0] == 'plants_hy':
        df['tracker_custom'] = 'GOGPT'

    return df


for df in gdf_to_be_merged:
  df = split_goget_ggit(df)
  print(df['tracker'].iloc[0])
  # print(df.columns)


term
pipes
plants
extraction
plants_hy


In [ ]:

def assign_conversion_factors(df, conversion_df):

    for row in df.index:
        print(df.columns)
        if df.loc[row, 'tracker_custom'] == 'GOGET-oil':
            df.loc[row, 'original_units'] = conversion_df[conversion_df['tracker']=='GOGET-oil']['original_units'].values[0]
            df.loc[row, 'conversion_factor'] = conversion_df[conversion_df['tracker']=='GOGET-oil']['conversion_factor'].values[0]

        elif df.loc[row, 'tracker_custom'] == 'GGIT-export':
            df.loc[row, 'original_units'] = conversion_df[conversion_df['tracker']=='GGIT-export']['original_units'].values[0]
            df.loc[row, 'conversion_factor'] = conversion_df[conversion_df['tracker']=='GGIT-export']['conversion_factor'].values[0]
        elif df.loc[row, 'tracker_custom'] == 'GGIT-import':
            df.loc[row, 'original_units'] = conversion_df[conversion_df['tracker']=='GGIT-import']['original_units'].values[0]
            df.loc[row, 'conversion_factor'] = conversion_df[conversion_df['tracker']=='GGIT-import']['conversion_factor'].values[0]

        elif df.loc[row, 'tracker_custom'] == 'GGIT':
            df['original_units'] = conversion_df[conversion_df['tracker']=='GGIT']['original_units'].values[0]
            df['conversion_factor'] = conversion_df[conversion_df['tracker']=='GGIT']['conversion_factor'].values[0]
        elif df.loc[row, 'tracker_custom'] == 'GOGPT':
            df['original_units'] = conversion_df[conversion_df['tracker']=='GOGPT']['original_units'].values[0]
            df['conversion_factor'] = conversion_df[conversion_df['tracker']=='GOGPT']['conversion_factor'].values[0]

    return df



for df in gdf_to_be_merged:
  df = assign_conversion_factors(df, conversion_df)
  print(df['tracker'].iloc[0])
  print(df.columns)
  if 'fuel-filter' not in df.columns:
    input('issue here')
  elif 'geometry' not in df.columns:
      input('issue here')

Streaming output truncated to the last 5000 lines.
      dtype='object')
Index(['unit-id', 'wiki-name', 'status', 'country', 'country-list',
       'subnational-unit-(province,-state)', 'gem-region', 'latitude',
       'longitude', 'location-accuracy', 'discovery-year', 'fid-year',
       'production-start-year', 'operator', 'owner', 'parent',
       'project-or-complex', 'production---oil-(million-bbl/y)',
       'production-year---oil', 'production---gas-(million-m³/y)',
       'production-year---gas',
       'production---total-(oil,-gas-and-hydrocarbons)-(million-boe/y)',
       'wiki-url', 'tracker-acro', 'official_name', 'country_to_check',
       'tracker', 'geometry', 'fuel-filter', 'maturity', 'tracker_custom',
       'original_units', 'conversion_factor'],
      dtype='object')
Index(['unit-id', 'wiki-name', 'status', 'country', 'country-list',
       'subnational-unit-(province,-state)', 'gem-region', 'latitude',
       'longitude', 'location-accuracy', 'discovery-year', 'fi

In [ ]:
# taken from all_config 1/30/25
renaming_cols_dict = {'plants': {'gem-location-id':'pid', 'gem-unit-id': 'id','wiki-url': 'url','country/area': 'areas', 'plant-name': 'name', 'unit-name': 'unit_name',
                                'capacity-(mw)': 'capacity', 'owner(s)': 'owner', 'parent(s)': 'parent', 'plant-name-in-local-language-/-script': 'other-local', 'other-name(s)': 'other-name',
                                'start-year': 'start_year', 'state/province': 'subnat'},

                      'plants_hy': {'gem-location-id':'pid', 'gem-unit-id': 'id','wiki-url': 'url','country/area': 'areas', 'plant-name': 'name', 'unit-name': 'unit_name',
                                'capacity-(mw)': 'capacity', 'owner(s)': 'owner', 'parent(s)': 'parent', 'plant-name-in-local-language-/-script': 'other-local', 'other-name(s)': 'other-name',
                                'start-year': 'start_year', 'state/province': 'subnat'},

                      'extraction': {'unit-id':'id', 'wiki-name': 'name', 'country': 'areas', 'subnational-unit-(province,-state)': 'subnat', 'discovery-year': 'start_year', 'production-start-year': 'prod_start_year',
                                'gem-region': 'region', 'wiki-url': 'url', 'production---oil-(million-bbl/y)': 'prod_oil', 'production---gas-(million-m³/y)': 'prod_gas',
                                'production---total-(oil,-gas-and-hydrocarbons)-(million-boe/y)': 'capacity','production-year---oil': 'prod_year_oil', 'production-year---gas': 'prod_year_gas',
                                'country-list':'mult_countries', 'fuel-type': 'fuel'},

                      'pipes': {'projectid':'id','countries': 'areas','wiki': 'url',
                                   'pipelinename':'name', 'segmentname':'unit_name',
                                   'startyear1': 'start_year', 'capacity': 'given_capacity','capacitybcm/y': 'capacity', 'startstate/province': 'subnat',
                                   'startregion': 'region', 'endstate/province': 'subnat2', 'endregion': 'region2', 'otherenglishnames': 'other-name',
                                    'otherlanguageprimarypipelinename': 'other-local',
                                   },
                      'term': {'comboid':'id','wiki': 'url', 'terminalname': 'name',
                                   'unitname': 'unit_name', 'country': 'areas', 'capacity': 'given_capacity','capacityinmtpa': 'capacity', 'startyear1': 'start_year', 'Region': 'region',
                                   'State/Province': 'subnat', 'otherlanguagename': 'other-name'},

                        }

final_cols = ['unit_id', 'mapname','tracker-acro','url', 'areas','name', 'unit_name', 'capacity', 'status', 'subnat', 'region', 'owner', 'parent', 'tracker', 'tracker_custom',
       'original_units', 'conversion_factor', 'geometry', 'river', 'area2', 'region2', 'subnat2', 'capacity1', 'capacity2',
       'prod-coal', 'pid','id', 'prod_oil', 'prod_gas', 'prod_year_oil', 'prod_year_gas', 'fuel', 'pci5', 'pci6', 'geometry', 'tracker', 'start_year', 'region2', 'subnat2', 'other-local', 'other-name',
       'fuel-filter', 'maturity', 'tracker_custom', 'original_units','mult_countries', 'prod_start_year',
       'conversion_factor', 'capacity', 'location', 'prefecture/district', 'state/province', 'search', 'operator(s)_search', 'name_search',
       'owner_search', 'parent_search']


def rename_gdfs(df):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace(' ', '-')

    tracker_sel = df['tracker'].iloc[0] # plants, term, pipes, extraction

    renaming_dict_sel = renaming_cols_dict[tracker_sel]
    # rename the columns!
    df = df.rename(columns=renaming_dict_sel)
    df.reset_index(drop=True, inplace=True)
    # find any duplicated column names
    all_columns = df.columns
    column_counts = Counter(all_columns)
    duplicate_columns = [name for name, count in column_counts.items() if count > 1]
    if len(duplicate_columns) > 0:
        print(f"Duplicate column names found: {duplicate_columns} for {tracker_sel}")
        input('FIX')
    else:
        print("No duplicate column names found.")
    print(duplicate_columns)  # Output: ['geometry', 'tracker', 'capacity', ...]
    # for i, col in enumerate(duplicate_columns):
    #     df.rename(columns={col: f"{col}_{i+1}"}, inplace=True)

    return df

final_list = []
for df in gdf_to_be_merged:
    df = rename_gdfs(df)
    final_list.append(df)
    print(df['tracker'].iloc[0])
    print(df.columns)





No duplicate column names found.
[]
term
Index(['terminalid', 'projectid', 'id', 'url', 'name', 'unit_name',
       'facilitytype', 'status', 'areas', 'other-name', 'fuel',
       'given_capacity', 'capacityunits', 'tempfacility', 'importexportonly',
       'lastupdated', 'otherenglishnames', 'owner', 'parent',
       'parenthqcountry', 'proposalyear', 'proposalmonth', 'constructionyear',
       'constructionmonth', 'start_year', 'startmonth1', 'startyear2',
       'startyear3', 'delayedstartyear', 'delayedstartmonth', 'delayed',
       'delaytype', 'startyearearliest', 'shelvedyear', 'cancelledyear',
       'stopyear', 'shelvedcancelledstatustype', 'capacity', 'capacityinbcm/y',
       'location', 'prefecture/district', 'state/province', 'region',
       'subregion', 'latitude', 'longitude', 'accuracy', 'powerplantssupplied',
       'cost', 'costunits', 'costusd', 'fidstatus', 'fidyear', 'offshore',
       'floating', 'floatingvesselname', 'lh2', 'nh3', 'syntheticlng',
       'retrofi

In [ ]:
# now merge


def merge_all_gdfs(final_list):

    # one_gdf = pd.concat(final_list, sort=False, verify_integrity=True, ignore_index=True)
    for df in final_list:
        # print(df.columns)
        print(df.index)
        df = df.reset_index()
        # print(len(df))
        # find any duplicate rows between all dfs in the final_list
        # df.drop_duplicates(inplace=True)
        # print(len(df))
        # input('check')
        # # print(df.index)
        if 'geometry' not in df.columns:
            df = convert_coords_to_point(df)

        # tracker = df['tracker'].iloc[0]
        # na_rows_df = df[df['geometry'].isna()]
        # print(len(na_rows_df))
        # na_rows_df.to_csv(f'/content/drive/MyDrive/output_from_colab/na-rows-{tracker}.csv')
        # input(f'check length of na rows for {tracker} before main merge')

    one_gdf = pd.concat(final_list, sort=False, ignore_index=True)

    # one_gdf.drop_duplicates(inplace=True)
    one_gdf.reset_index(drop=True, inplace=True)


    # print(one_gdf.columns)
    # na_rows_one_gdf = one_gdf[one_gdf['geometry'].isna()]
    # print(len(na_rows_one_gdf))
    # na_rows_one_gdf.to_csv(f'/content/drive/MyDrive/output_from_colab/na-rows-{na_rows_one_gdf}.csv')
    # input(f'check length of na rowsafter main merge')
    return one_gdf

one_gdf = merge_all_gdfs(final_list)
# one_gdf['areas']

# make sure the gogpt hy row stays and the other one ids removed with drop duplicate

# sort by tracker so we know if gogpt-hy or gogpt is first
one_gdf = one_gdf.sort_values(by='tracker', ascending=True, axis=0)
print(set(one_gdf['tracker'].to_list()))
# check how many total count by tracker after drop duplicates
print(one_gdf[['id', 'tracker']].groupby('tracker').count())

one_gdf.drop_duplicates(subset='id', inplace=True, keep='last') # add logic so it defaults to keeping the gogpt-hy ones over the gogpt ones, so if yes in gogpt data remove
one_gdf.reset_index(drop=True, inplace=True)
# filter down one_gdf to just id
print(one_gdf[['id', 'tracker']].groupby('tracker').count())



RangeIndex(start=0, stop=158, step=1)
RangeIndex(start=0, stop=1272, step=1)
RangeIndex(start=0, stop=2084, step=1)
RangeIndex(start=0, stop=927, step=1)
RangeIndex(start=0, stop=110, step=1)
{'plants', 'extraction', 'term', 'plants_hy', 'pipes'}
              id
tracker         
extraction   927
pipes       1272
plants      2084
plants_hy    110
term         158
              id
tracker         
extraction   927
pipes       1272
plants      2068
plants_hy    110
term         158


In [ ]:
len(one_gdf)
if 'fuel-filter' not in one_gdf.columns:
    print(one_gdf.columns)


In [ ]:
renamed_one_gdf_by_map = {'europe': one_gdf}

In [ ]:
# create_search_column
diacritic_map = {
    'a': ["a", "á", "à", "â", "ã", "ä", "å"],
    'e': ["e", "é", "è", "ê", "ë"],
    'i': ["i", "í", "ì", "î", "ï"],
    'o': ["o", "ó", "ò", "ô", "õ", "ö", "ø"],
    'u': ["u", "ú", "ù", "û", "ü"],
    'c': ["c", "ç"],
    'n': ["n", "ñ"],
}


def remove_diacritics(name_value):
    # name_value = name_value.fillna('')
    no_diacritics_name_value = ''
    if type(name_value) != float:
        no_diacritics_name_value = name_value[:]
        for char in no_diacritics_name_value:
            for k, v in diacritic_map.items():
                if char in v:
                    no_diacritics_name_value = no_diacritics_name_value.replace(char, k)

    return no_diacritics_name_value

def create_search_column(dict_of_gdfs):
    # this creates a new column for project and project in local language
    # in the column it'll be removed of any diacritics
    # this allows for quick searching
    #     for mapname, one_gdf in cleaned_dict_map_by_one_gdf.items():
    dict_of_gdfs_with_search = {}
    for mapname, one_gdf in dict_of_gdfs.items():

        print('testing create_search_column with no diacritics for first time')
        col_names = ['plant-name', 'parent(s)', 'owner(s)', 'operator(s)', 'name', 'owner', 'parent']
        one_gdf['search'] = '' # add all search items as one string to this column
        for col in col_names:
            if col in one_gdf.columns:
                # print(one_gdf[col].head(10))
                new_col_name = f'{col}_search'
                one_gdf[new_col_name] = one_gdf[col].apply(lambda x: remove_diacritics(x))
                one_gdf['search'] = one_gdf['search'] + ' ' + one_gdf[new_col_name]
                # print(one_gdf[new_col_name].head(10))

        dict_of_gdfs_with_search[mapname] = one_gdf

        print(dict_of_gdfs_with_search.keys)
        print('above are keys in dict_of_gdfs_with_search')
    return dict_of_gdfs_with_search

In [ ]:
gas_only_maps = ['AGT', 'EGT', 'asia', 'europe', 'ggit']

def conversion_multiply(row):
    cap = float(row['cleaned_cap'])
    factor = float(row['conversion_factor'])
    # print(f'this is factor! {factor}')

    result = float(cap * factor)
    # print(f'this is result! {result}')
    return result

def is_number(n):
    is_number = True
    try:
        num = float(n)
        # check for "nan" floats
        is_number = num == num   # or use `math.isnan(num)`
    except ValueError:
        is_number = False
    return is_number

def check_and_convert_float(x):
    if is_number(x):
        return float(x)
    else:
        return np.nan
def workaround_table_float_cap(row, cap_col):
    cap = row[cap_col]
    cap = check_and_convert_float(cap)
    if isinstance(cap, (int, float)):
        cap = float((round(cap, 3))) # handle rounding and converting from string to float to round later
    else:
        print(f'issue cap should be a float')

    return cap

def workaround_table_units(row):

    units_of_m = str(row['original_units'])

    return units_of_m


def capacity_conversions(cleaned_dict_map_by_one_gdf):

# you could multiply all the capacity/production values in each tracker by the values in column C,
# "conversion factor (capacity/production to common energy equivalents, TJ/y)."
# For this to work, we have to be sure we're using values from each tracker that are standardized
# to the same units that are stated in this sheet (column B, "original units").

# need to get nuances for each tracker, it's just GGIT if point then do lng terminal if line then do pipeline
    cleaned_dict_map_by_one_gdf_with_conversions = {}
    for mapname, one_gdf in cleaned_dict_map_by_one_gdf.items():
        gdf_converted = one_gdf.copy()
        if mapname in gas_only_maps:
            print('no need to handle for hydro having two capacities')
            print(gdf_converted.columns)
            # continue
        else:
            # first let's get GHPT cap added
            # # printmapname) # africa
            # # printset(gdf_converted['tracker-acro'].to_list())) # only pipeline
            ghpt_only = gdf_converted[gdf_converted['tracker-acro']=='GHPT'] # for GGPT we need to re run it to get it
            print(len(ghpt_only))
            print(mapname)
            # ##(input('check')
            gdf_converted = gdf_converted[gdf_converted['tracker-acro']!='GHPT']
            # # printlen(ghpt_only['capacity']))
            # # printlen(ghpt_only['capacity1']))
            # # printlen(ghpt_only['capacity2']))
            # # # ##(input('Check that they are all equal GHPT')
            ghpt_only['capacity'] = ghpt_only.apply(lambda row: row['capacity'] + row['capacity2'], axis=1)
            gdf_converted = pd.concat([gdf_converted, ghpt_only],sort=False).reset_index(drop=True)
        # # # printlen(gdf_converted))

        gdf_converted['cleaned_cap'] = pd.to_numeric(gdf_converted['capacity'], errors='coerce')

        total_counts_trackers = []
        avg_trackers = []
        print(gdf_converted.columns)
        # print(f"this is all trackers: {set(gdf_converted['tracker-acro'].to_list())}")

        # gdf_converted['tracker-acro'] = gdf_converted['tracker-acro']
        # # printf"this is all trackers: {set(gdf_converted['tracker-acro'].to_list())}")

        gdf_converted = gdf_converted[gdf_converted['tracker-acro']!=''] # new to filter out nan


        for tracker in set(gdf_converted['tracker-acro'].to_list()):
            # for singular map will only go through one
            total = len(gdf_converted[gdf_converted['tracker-acro'] == tracker])
            sum = gdf_converted[gdf_converted['tracker-acro'] == tracker]['cleaned_cap'].sum()
            avg = sum / total
            total_pair = (tracker, total)
            total_counts_trackers.append(total_pair)
            avg_pair = (tracker, avg)
            avg_trackers.append(avg_pair)

        for row in gdf_converted.index:
            cap_cleaned = gdf_converted.loc[row, 'cleaned_cap']
            tracker = gdf_converted.loc[row, 'tracker-acro']
            if pd.isna(cap_cleaned):
                for pair in avg_trackers:
                    if pair[0] == tracker:
                        gdf_converted.loc[row, 'cleaned_cap'] = pair[1]
            cap_cleaned = gdf_converted.loc[row, 'cleaned_cap']
            # if pd.isna(cap_cleaned):
                # print'still na')
                # # ##(input('still na')


        pd.options.display.float_format = '{:.0f}'.format
        # gdf_converted['ea_scaling_capacity'] = gdf_converted.apply(lambda row: conversion_equal_area(row), axis=1) # square root(4 * capacity / pi)

        gdf_converted['scaling_capacity'] = gdf_converted.apply(lambda row: conversion_multiply(row), axis=1)
        # must be float for table to sort
        gdf_converted['capacity-table'] = gdf_converted.apply(lambda row: pd.Series(workaround_table_float_cap(row, 'capacity')), axis=1)
        gdf_converted['units-of-m'] = gdf_converted.apply(lambda row: pd.Series(workaround_table_units(row)), axis=1)
        # gdf_converted['units-of-m'] = gdf_converted.apply(lambda row: '' if 'GOGET' in row['tracker-acro'] else row['units-of-m'], axis=1)

        # below doesn't work cap details was empty all the time
        # gdf_converted = workaround_no_sum_cap_project(gdf_converted) # adds capacity-details for singular maps we can just disregard
        # TODO nov 13 test this I think it now adds all cap for a project and applies the original units to it
        # gdf_converted['capacity-details-unit'] = gdf_converted.apply(lambda row: workaround_display_cap(row, 'capacity-details'), axis=1)

        cleaned_dict_map_by_one_gdf_with_conversions[mapname] = gdf_converted


    # # printcleaned_dict_map_by_one_gdf_with_conversions.keys())
    # # # ##(input('check that there are enough maps')
    return cleaned_dict_map_by_one_gdf_with_conversions


In [ ]:
def fix_status_inferred(df):
    # print(f"Statuses before: {set(df['status'].to_list())}")
    inferred_statuses_cancelled = df['status'].str.contains('cancelled - inferred')
    inferred_statuses_shelved = df['status'].str.contains('shelved - inferred')


    df.loc[inferred_statuses_cancelled, 'status'] = 'cancelled'
    df.loc[inferred_statuses_shelved,'status'] = 'shelved'


    return df

def map_ready_statuses(cleaned_dict_map_by_one_gdf_with_conversions):
    cleaned_dict_by_map_one_gdf_with_better_statuses = {}
    for mapname, gdf in cleaned_dict_map_by_one_gdf_with_conversions.items():
        path_for_test_results = '/content/drive/MyDrive/output_from_colab/'

        # print(set(gdf['status'].to_list()))
        mask_gbpt = gdf['tracker-acro'] == 'GBPT'
        # print(gdf.loc[mask_gbpt, 'status'])
        # #(input('check statuses of bio before')

        gdf['status'] = gdf['status'].fillna('Not Found') # ValueError: Cannot mask with non-boolean array containing NA / NaN values
        gdf['status'] = gdf['status'].replace('', 'Not Found') # ValueError: Cannot mask with non-boolean array containing NA / NaN values
        # print(set(gdf['status'].to_list()))
        gdf_map_ready = fix_status_inferred(gdf)

        # Create masks for the 'tracker-acro' conditions
        mask_gcmt = gdf_map_ready['tracker-acro'] == 'GCMT'
        mask_goget = gdf_map_ready['tracker-acro'] == 'GOGET'
        mask_gbpt = gdf_map_ready['tracker-acro'] == 'GBPT'

        # Update 'status' to 'Retired' where both masks are True
        gdf_map_ready['status'].fillna('', inplace=True)
        mask_status_empty = gdf_map_ready['status'] == ''
        # print(mask_status_empty)
        # # #(input('check the above for empty statuses')
        # print(gdf_map_ready.loc[mask_gbpt, 'status'])
        # #(input('check statuses of bio')

        # Update 'status' to 'Not Found' where both masks are True
        gdf_map_ready.loc[mask_status_empty & mask_gcmt, 'status'] = 'retired'
        gdf_map_ready.loc[mask_status_empty & mask_goget, 'status'] = 'not found'
        gdf_map_ready['status_legend'] = gdf_map_ready.copy()['status'].str.lower().replace({
                    # proposed_plus
                    'proposed': 'proposed_plus',
                    'announced': 'proposed_plus',
                    'discovered': 'proposed_plus',
                    # pre-construction_plus
                    'pre-construction': 'pre-construction_plus',
                    'pre-permit': 'pre-construction_plus',
                    'permitted': 'pre-construction_plus',
                    # construction_plus
                    'construction': 'construction_plus',
                    'in development': 'construction_plus',
                    # mothballed
                    'mothballed': 'mothballed_plus',
                    'idle': 'mothballed_plus',
                    'shut in': 'mothballed_plus',
                    # retired
                    'retired': 'retired_plus',
                    'closed': 'retired_plus',
                    'decommissioned': 'retired_plus',
                    'not found': 'not-found'})


        # Create a mask for rows where 'status' is empty

        gdf_map_ready_no_status = gdf_map_ready.loc[mask_status_empty]
        # # # ##(input(f'check no status df: {gdf_map_ready_no_status}')

        gdf_map_ready_no_status.to_csv(f'/content/drive/MyDrive/output_from_colab/no-status-{mapname}.csv')
        # make sure all statuses align with no space rule
        # gdf_map_ready['status'] = gdf_map_ready['status'].apply(lambda x: x.strip().replace(' ','-'))
        gdf_map_ready['status_legend'] = gdf_map_ready['status_legend'].apply(lambda x: x.strip().replace('_','-'))
        # # printset(gdf_map_ready['status'].to_list()))
        gdf_map_ready['status'] = gdf_map_ready['status'].apply(lambda x: x.lower())
        # # printset(gdf_map_ready['status'].to_list()))

        cleaned_dict_by_map_one_gdf_with_better_statuses[mapname] = gdf_map_ready

    # # printcleaned_dict_by_map_one_gdf_with_better_statuses.keys())
    # # # ##(input('check that there are enough maps')
    return cleaned_dict_by_map_one_gdf_with_better_statuses




In [ ]:

def map_ready_countries(cleaned_dict_by_map_one_gdf_with_better_statuses):
    cleaned_dict_by_map_one_gdf_with_better_countries = {}
    for mapname, gdf in cleaned_dict_by_map_one_gdf_with_better_statuses.items():
        print(f'We are on mapname: {mapname}')

        # check that areas isn't empty
        tracker_sel = gdf['tracker-acro'].iloc[0]

        gdf['areas'] = gdf['areas'].fillna('')

        empty_areas = gdf[gdf['areas']=='']
        if len(empty_areas) > 0:
            print(f'Check out which rows are empty for countries for map: {mapname}')
            print(empty_areas)
            # #(input('Remove above')
            empty_areas.to_csv(f'/content/drive/MyDrive/output_from_colab/empty-areas-{tracker_sel}.csv')

        # this formats subnational area for detail maps
        # we would also want to overwrite the subnat and say nothing ""
        gdf['count-of-semi'] = gdf.apply(lambda row: row['areas'].strip().split(';'), axis=1) # if len of list is more than 2, then split more than once
        gdf['count-of-semi'] = gdf.apply(lambda row: row['areas'].strip().split('-'), axis=1) # for goget
        gdf['count-of-semi'] = gdf.apply(lambda row: row['areas'].strip().split(','), axis=1) # just adding in case

        gdf['multi-country'] = gdf.apply(lambda row: 't' if len(row['count-of-semi']) > 1 else 'f', axis=1)
        # # printgdf['multi-country'])
        # if t then make areas-display
        gdf['subnat'].fillna('', inplace=True)
        # if one country and subnat exists TEST THIS
        gdf['areas-subnat-sat-display'] = gdf.apply(lambda row: f"{row['subnat'].strip().strip('')}, {row['areas'].strip().strip('')}" if row['multi-country'] == 'f' and row['subnat'] != '' else row['areas'].strip(), axis=1) # row['areas'].strip()

        maskt = gdf['multi-country']=='t'

        gdf.loc[maskt, 'areas-subnat-sat-display'] = 'multiple areas/countries'

        if mapname in gas_only_maps:
            # handle for gas only maps
            print('In not map only area of function')
            gdf['areas'] = gdf['areas'].fillna('')
            gdf['areas'] = gdf['areas'].str.replace(',', ';')
            gdf['areas'] = gdf['areas'].apply(lambda x: f"{x.strip()};")
            print(gdf['areas'])
            # input('check above has semicolon')

        else:

            # print(gdf_map_ready[['areas', 'tracker-acro', 'name']])
            # print(set(gdf_map_ready['areas'].to_list()))
            # print(set(gdf_map_ready['area2'].to_list()))
            gdf['area2'] = gdf['area2'].fillna('')
            gdf['areas'] = gdf['areas'].fillna('')
            # print(set(gdf['areas'].to_list()))
            # print(set(gdf['area2'].to_list()))
            nan_areas = gdf[gdf['areas']=='']
            print(f'Nan areas: {len(nan_areas)}')
            # print(nan_areas)
            input('check nan areas')
            issues = []
            tracker_issues = []
            for row in gdf.index:
                if gdf.loc[row, 'areas'] == '':
                    issues.append(row)
                    tracker_issues.append(gdf.loc[row, 'tracker-acro'])
            # if len(issues) >0 :
            #     print(f'No areas here for these trackers:')
            #     print(set(tracker_issues))
            gdf = gdf.drop(issues)
            issues_df = {'missing_country': issues}
            issues_df = pd.DataFrame(issues_df)
            issues_df.to_csv(f'/content/drive/MyDrive/output_from_colab/missing_county{mapname}.csv')
            print('Printed issues_df to file and dropped themf rom the df.')
            # ##(input('All area2s in gdf check if any are numbers')
            if mapname == 'Global':
                gdf['areas'] = f"{gdf['areas']};"

            else:
                for row in gdf.index:

                    if gdf.loc[row, 'area2'] != '':

                        gdf.at[row, 'areas'] = f"{gdf.loc[row, 'areas'].strip()};{gdf.loc[row, 'area2'].strip()};"
                        # print(f"Found a area2! Hydro? {gdf.loc[row, 'areas']} {gdf.loc[row, 'tracker-acro']} {gdf.loc[row, 'name']}")

                    else:
                        # make it so all areas even just one end with a semincolon
                        gdf['areas'] = gdf['areas'].fillna('')
                        # nan_areas = gdf[gdf['areas']=='']
                        # print(f'in else: {set(gdf["areas"].to_list())}') # find the rows that are nan or float
                        ser = gdf['areas']
                        try:
                            ser_str = ser.astype(str)
                        except:
                            for row in ser.index:
                                val = ser.iloc[row]
                                try:
                                    val_str = str(val)
                                except:
                                    print("Error!" + f" val couldn't be converted to str: {val}")

                        gdf.at[row, 'areas'] = f"{gdf.loc[row, 'areas'].strip()};"
        if 'fuel-filter' not in gdf.columns:
            input('issue here')
        cleaned_dict_by_map_one_gdf_with_better_countries[mapname] = gdf

    return cleaned_dict_by_map_one_gdf_with_better_countries


In [ ]:

def workarounds_eg_interim_goget_gcmt(cleaned_dict_by_map_one_gdf_with_better_countries):
    # this function mostly creates a new col for correctly formatted info when there is multiple countries, especially for the details card
    # it also handles oil and gas for goget, TODO should add removal of oil for gas only map maybe?
    one_gdf_by_maptype = {}
    for mapname, gdf in cleaned_dict_by_map_one_gdf_with_better_countries.items():
        gdf = gdf.copy()

        list_invalid_goget = []
        list_invalid_gcmt = []
        if mapname in ['GIPT', 'Global']:
            # does not have goget
            one_gdf_by_maptype[mapname] = gdf

        else:
            # # # printgdf.columns)
            # # # ##(input('Check if prod-oil is there in columns')
            for row in gdf.index:

                tracker = (gdf.loc[row, 'tracker'])
                #if goget then make capacity table and capacity details empty


                if tracker == 'extraction':
                    gdf.loc[row, 'capacity-table'] = np.nan
                    gdf.loc[row, 'capacity-details'] = ''
                    prod_oil = gdf.loc[row, 'prod_oil']
                    prod_gas = gdf.loc[row, 'prod_gas']
                    prod_oil = check_and_convert_float(prod_oil) # NEW TO TEST FOR ROUNDING ALL GETTING CAUGH TIN INVALID GOGET
                    prod_gas = check_and_convert_float(prod_gas)

                else:
                    continue
        one_gdf_by_maptype[mapname] = gdf
    # # printone_gdf_by_maptype.keys())
    return one_gdf_by_maptype


In [ ]:
def check_in_range(value, min_val, max_val):
    # doesn't handle na because already was handled
    if min_val <= value <= max_val:

        return value

    else:
        print('problem with coords:')
        print(f'value:{value}, min_val:{min_val}, max_val:{max_val}')
        return np.nan


def pci_eu_map_read(gdf):
    # take columns PCI5 and PCI6
    # create one column, both, 5, 6, none, all as strings

    gdf['pci-list'] = ''
    for row in gdf.index:
        pci5 = gdf.loc[row, 'pci5']
        pci6 = gdf.loc[row, 'pci6']
        if pci5 == 'yes' and pci6 == 'yes':
            gdf.at[row, 'pci-list'] = 'both'
        elif pci5 == 'yes':
            gdf.at[row, 'pci-list'] = '5'
        elif pci6 == 'yes':
            gdf.at[row, 'pci-list'] = '6'
        else:
            gdf.at[row, 'pci-list'] = 'none'

    return gdf

tracker_to_fullname = {
                    "GCPT": "coal power station",
                    "GOGPT": "oil & gas power station",
                    "GBPT": "bioenergy power station",
                    "GNPT": "nuclear power plant",
                    "GSPT": "solar power plant",  # GSPT is used for both "solar thermal" and "solar PV"
                    "GWPT": "wind power plant",
                    "GHPT": "hydropower plant",
                    "GGPT": "geothermal power plant",
                    "GOGET-oil": "oil & gas extraction area",
                    # "GOGET - gas": "gas extraction area",
                    "GOIT": "oil pipeline",
                    "GGIT-eu": "gas pipeline",
                    "GGIT": "gas pipeline",
                    "GGIT-import": "LNG import terminal",
                    "GGIT-export": "LNG export terminal",
                    "GCMT": "coal mine",
                    "GCTT": "coal terminal"
}


tracker_to_legendname = {
                    "GCPT": "coal-power-station",
                    "GOGPT": "oil-gas-power-station",
                    "GBPT": "bioenergy-power-station",
                    "GNPT": "nuclear-power-plant",
                    "GSPT": "solar-power-plant",  # GSPT is used for both "solar thermal" and "solar PV"
                    "GWPT": "wind-power-plant",
                    "GHPT": "hydropower-plant",
                    "GGPT": "geothermal-power-plant",
                    "GOGET-oil": "oil & gas extraction area",
                    # "GOGET - gas": "gas-extraction-area",
                    "GOIT": "oil-pipeline",
                    "GGIT": "gas-pipeline",
                    "GGIT-import": "LNG-import-terminal",
                    "GGIT-export": "LNG-export-terminal",
                    "GCMT": "coal-mine",
                    "GCTT": "coal-terminal"
}


def coordinate_qc(df):
    issues_coords = {} # acro, df line
    df = df.reset_index()
    tracker = df['tracker'].loc[0]


    if 'WKTFormat' in df.columns: # special for hydrogen europe file
        # df['wktformat_clean'] = df['Route'].apply(lambda x: check_and_convert_float(x))
        pass


    else:
        df['float_col_clean_lat'] = df['latitude'].apply(lambda x: check_and_convert_float(x))

        df['float_col_clean_lng'] = df['longitude'].apply(lambda x: check_and_convert_float(x))

        for row in df.index:
            if pd.isna(df.loc[row, 'float_col_clean_lat']):
                issues_coords[tracker] = df.loc[row]
                df.drop(index=row, inplace=True)
            elif pd.isna(df.loc[row, 'float_col_clean_lng']):
                issues_coords[tracker] = df.loc[row]
                df.drop(index=row, inplace=True)


        # check that the numbers fall within a range
        # QC
        acceptable_range = {
            'lat': {'min': -90, 'max': 90},
            'lng': {'min': -180, 'max': 180}
        }

        df['float_col_clean_lat'] = df['float_col_clean_lat'].apply(
            lambda x: check_in_range(x, acceptable_range['lat']['min'], acceptable_range['lat']['max'])
        )
        # print(df['float_col_clean_lat'])
        # input('check after check in range or nan')
        df['float_col_clean_lng'] = df['float_col_clean_lng'].apply(
            lambda x: check_in_range(x, acceptable_range['lng']['min'], acceptable_range['lng']['max'])
        )
        # check_in_range could return nan if out of range so need to drop nans
        # this should have been handled in QC the range stuff
        for row in df.index:
            if pd.isna(df.loc[row, 'float_col_clean_lat']):
                print(df.loc[row])
                issues_coords[tracker] = df.loc[row]
                df.drop(index=row, inplace=True)
            elif pd.isna(df.loc[row, 'float_col_clean_lng']):
                print(df.loc[row])
                issues_coords[tracker] = df.loc[row]
                df.drop(index=row, inplace=True)


            else:
                df.loc[row, 'latitude'] = df.loc[row, 'float_col_clean_lat']
                df.loc[row, 'longitude'] = df.loc[row, 'float_col_clean_lng']

    return df, issues_coords



def last_min_fixes(one_gdf_by_maptype):
    one_gdf_by_maptype_fixed = {}
    # # printone_gdf_by_maptype.keys())
    # # # ##(input('check that GIPT is in the above')
    for mapname, gdf in one_gdf_by_maptype.items():            # do filter out oil
        gdf = gdf.copy()
        # handle situation where Guinea-Bissau IS official and ok not to be split into separate countries
        gdf['areas'] = gdf['areas'].apply(lambda x: x.replace('Guinea,Bissau','Guinea-Bissau'))

        # something is happening when we concat, we lose goget's name ...
        # gdf_empty_name = gdf[gdf['name']=='']
        # # # printf"This is cols for gdf: {gdf.columns}")
        gdf['name'].fillna('',inplace=True)
        gdf['name'] = gdf['name'].astype(str)
        gdf['capacity'].fillna('',inplace=True)

        gdf['wiki-from-name'] = gdf.apply(lambda row: f"https://www.gem.wiki/{row['name'].strip().replace(' ', '_')}", axis=1)
        # row['url'] if row['url'] != '' else
        # handles for empty url rows and also non wiki cases SHOULD QC FOR THIS BEFOREHAND!! TO QC
        # need to switch to '' not nan so can use not in
        gdf['url'].fillna('',inplace=True)
        gdf['url'] = gdf.apply(lambda row: row['wiki-from-name'] if 'gem.wiki' not in row['url'] else row['url'], axis=1)


        gdf.columns = [col.replace('_', '-') for col in gdf.columns]
        gdf.columns = [col.replace('  ', ' ') for col in gdf.columns]
        gdf.columns = [col.replace(' ', '-') for col in gdf.columns]


        if 'fuel-filter' not in gdf.columns:
            input('issue here')
        # gdf['tracker-acro'] = gdf['tracker-acro']
        # let's also look into empty url, by tracker I can assign a filler

        # gdf['url'] = gdf['url'].apply(lambda row: row[filler_url] if row['url'] == '' else row['url'])

        # gdf['capacity'] = gdf['capacity'].apply(lambda x: x.replace('--', '')) # can't do that ...

        # translate acronyms to full names

        gdf['tracker-display'] = gdf['tracker-custom'].map(tracker_to_fullname)
        gdf['tracker-legend'] = gdf['tracker-custom'].map(tracker_to_legendname)
        # # # printset(gdf['tracker-display'].to_list()))
        # # # printf'Figure out capacity dtype: {gdf.dtypes}')
        gdf['capacity'] = gdf['capacity'].apply(lambda x: str(x).replace('--', ''))
        gdf['capacity'] = gdf['capacity'].apply(lambda x: str(x).replace('*', ''))

        # remove units-of-m if no capacity value ... goget in particular
        for row in gdf.index:
          if gdf.loc[row, 'capacity'] == '':
              gdf.loc[row, 'units-of-m'] = ''
          elif gdf.loc[row, 'capacity-details'] == '':
              gdf.loc[row, 'units-of-m'] = ''
          elif gdf.loc[row, 'capacity-table'] == np.nan:
              gdf.loc[row, 'units-of-m'] = ''


        # print all na rows to csv
        # na_rows = gdf[gdf['geometry'].isna()]
        # print(len(na_rows))
        # na_rows.to_csv(f'/content/drive/MyDrive/output_from_colab/na-rows-{mapname}.csv')
        # input('check length of na rows')
        # for row in na_rows.index:
        #     # find situations where lat and lng are both null
        #     lat = gdf.loc[row, 'latitude']
        #     lng = gdf.loc[row, 'longitude']
        #     if pd.isna(lat) and pd.isna(lng):
        #       print(row)

        test = gdf[gdf['name']=='Montoir LNG Terminal']
        print(test['geometry'])
        print(len(gdf))
        gdf.dropna(subset=['geometry'],inplace=True)
        print(len(gdf))
        # gdf_null_geo = gdf[gdf['geometry']== 'null']
        gdf = gdf[gdf.geometry.notnull()]
        # drop null geo
        # gdf.drop(gdf_null_geo.index, inplace=True)
        # print(len(gdf_null_geo))
        print(len(gdf))
        # input('check after dropped null')

        df_map, issues_coords = coordinate_qc(gdf)

        # issues_coords.to_df.to_csv(f'/content/drive/MyDrive/output_from_colab/issue_coords.csv')
        # put dict to df to csv
        issues_coords_df = pd.DataFrame.from_dict(issues_coords, orient='index')
        issues_coords_df.to_csv(f'/content/drive/MyDrive/output_from_colab/issue_coords.csv')

        year_cols = ['start-year', 'prod-year-gas']

        for col in year_cols:
            gdf[col] = gdf[col].apply(lambda x: str(x).split('.')[0])
            gdf[col].replace('-1', '')
        # TODO add check that year isn't a -1 ... goget.. for egt

        if mapname == 'europe':
            # print(mapname)
            gdf = pci_eu_map_read(gdf)
            # gdf = assign_eu_hydrogen_legend(gdf)
            # gdf = manual_lng_pci_eu_temp_fix(gdf)
            # gdf = swap_gas_methane(gdf)

        one_gdf_by_maptype_fixed[mapname] = gdf

    return one_gdf_by_maptype_fixed


In [ ]:
today_date = datetime.today()
finalcols = ['status-legend','status', 'fuel-filter', 'tracker-custom', 'maturity', 'pci-list', 'scaling-capacity', 'name', 'areas',
        'unit-name', 'owner', 'parent', 'capacity-table', 'units-of-m','start-year', 'prod-gas', 'prod-year-gas', 'tracker-display',
        'areas-subnat-sat-display','other-name', 'local-name', 'geometry', 'url', 'id', 'pid']

# Format the date in ISO format
iso_today_date = today_date.isoformat().split('T')[0]

def check_for_lists(gdf):
    print('in check for lists')
    print(gdf['fuel-filter'])
    for col in gdf.columns:
        if any(isinstance(val, list) for val in gdf[col]):
            print('Column: {0}, has a list in it'.format(col))
        else:
            pass

def make_sure_methane_is_none_maturity(gdf):
    print('in maturity')

    # gdf['maturity'] = gdf.apply(lambda row: 'none' if row['fuel-filter'] == 'methane' else row['maturity'])

    return gdf

def create_map_file(one_gdf_by_maptype_fixed):
    final_dict_gdfs = {}
    for mapname, gdf in one_gdf_by_maptype_fixed.items():
        # gdf = gdf.copy()
        if 'fuel-filter' not in gdf.columns:
            input('issue here')
        elif 'geometry' not in gdf.columns:
            input('issue here')

        # filter by finalcols
        cols_to_drop = [col for col in gdf.columns if col not in (finalcols + final_cols)]
        print(gdf.columns)

        gdf.drop(columns=cols_to_drop, inplace=True)

        print(gdf.columns)
        if 'fuel-filter' not in gdf.columns:
            input('issue here')
        elif 'geometry' not in gdf.columns:
            input('issue here')
        print(mapname)
        print(f'Saving file for map {mapname}')
        print(f'This is len of gdf {len(gdf)}')
        path_for_download_and_map_files = '/content/drive/MyDrive/output_from_colab/'

        gdf.to_file("/tmp/output.geojson", driver="GeoJSON")
        # if mapname in gas_only_maps: # will probably end up making all regional maps all energy I would think
        #     gdf = gdf.drop(['count-of-semi', 'multi-country', 'original-units', 'conversion-factor', 'cleaned-cap', 'wiki-from-name', 'tracker-legend'], axis=1) # 'multi-country', 'original-units', 'conversion-factor', 'cleaned-cap', 'wiki-from-name', 'tracker-legend']

        # else:
        #     gdf = gdf.drop(['count-of-semi','multi-country', 'original-units', 'conversion-factor', 'area2', 'region2', 'subnat2', 'capacity2', 'cleaned-cap', 'wiki-from-name', 'tracker-legend'], axis=1) #  'multi-country', 'original-units', 'conversion-factor', 'area2', 'region2', 'subnat2', 'capacity1', 'capacity2', 'cleaned-cap', 'wiki-from-name', 'tracker-legend']

        check_for_lists(gdf)

        print(gdf['geometry'])
        # df.set_geometry(gpd.GeoSeries.from_wkt(df['geometry']), inplace=True)
        # gdf = gdf.to_crs(epsg=4326)
        print(f'this is fuel filter {gdf["fuel-filter"]}')
        gdf = make_sure_methane_is_none_maturity(gdf)
        gdf.to_file(f'{path_for_download_and_map_files}{mapname}_{iso_today_date}.geojson', driver='GeoJSON', encoding='utf-8')
        gdf.to_csv(f'{path_for_download_and_map_files}{mapname}_{iso_today_date}.csv', encoding='utf-8')

        final_dict_gdfs[mapname] = gdf

    # print(final_dict_gdfs.keys())
    return final_dict_gdfs


In [ ]:

  renamed_one_gdf_by_map_with_search = create_search_column(renamed_one_gdf_by_map)

  cleaned_dict_map_by_one_gdf_with_conversions = capacity_conversions(renamed_one_gdf_by_map_with_search)

  cleaned_dict_by_map_one_gdf_with_better_statuses = map_ready_statuses(cleaned_dict_map_by_one_gdf_with_conversions)
  cleaned_dict_by_map_one_gdf_with_better_countries = map_ready_countries(cleaned_dict_by_map_one_gdf_with_better_statuses)
  one_gdf_by_maptype = workarounds_eg_interim_goget_gcmt(cleaned_dict_by_map_one_gdf_with_better_countries)
  one_gdf_by_maptype_fixed = last_min_fixes(one_gdf_by_maptype)
  final_dict_gdfs = create_map_file(one_gdf_by_maptype_fixed)



testing create_search_column with no diacritics for first time
<built-in method keys of dict object at 0x7b16e4e2ba80>
above are keys in dict_of_gdfs_with_search
no need to handle for hydro having two capacities
Index(['terminalid', 'projectid', 'id', 'url', 'name', 'unit_name',
       'facilitytype', 'status', 'areas', 'other-name',
       ...
       'mou-for-h2-supply?', 'contract-for-h2-supply?',
       'financing-for-supply-of-h2?',
       'co-located-with-electrolyzer/h2-production-facility?',
       'what-%-of-h2-blending-currently?', 'search', 'operator(s)_search',
       'name_search', 'owner_search', 'parent_search'],
      dtype='object', length=149)
Index(['terminalid', 'projectid', 'id', 'url', 'name', 'unit_name',
       'facilitytype', 'status', 'areas', 'other-name',
       ...
       'contract-for-h2-supply?', 'financing-for-supply-of-h2?',
       'co-located-with-electrolyzer/h2-production-facility?',
       'what-%-of-h2-blending-currently?', 'search', 'operator(s)_se

<ipython-input-128-eb7947e788cf>:93: RuntimeWarning: invalid value encountered in scalar divide
  avg = sum / total
<ipython-input-129-612e5e5205fb>:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdf_map_ready['status'].fillna('', inplace=True)


We are on mapname: europe


<ipython-input-130-8153dc6ec6a4>:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdf['subnat'].fillna('', inplace=True)


In not map only area of function
0              Ukraine;
1       United Kingdom;
2       United Kingdom;
3       United Kingdom;
4       United Kingdom;
             ...       
4530             Italy;
4531         Gibraltar;
4532            Greece;
4533           Ireland;
4534           Albania;
Name: areas, Length: 4535, dtype: object


<ipython-input-132-4db3c4757f32>:146: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  gdf['name'].fillna('',inplace=True)
<ipython-input-132-4db3c4757f32>:148: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.m

4437    POINT (-2.14278 47.30296)
4438    POINT (-2.14278 47.30296)
Name: geometry, dtype: geometry
4535
4535
4535


/usr/local/lib/python3.11/dist-packages/geopandas/geoseries.py:860: UserWarning: GeoSeries.notna() previously returned False for both missing (None) and empty geometries. Now, it only returns False for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use '~s.is_empty & s.notna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.notna', UserWarning)
  return self.notna()


Index(['terminalid', 'projectid', 'id', 'url', 'name', 'unit-name',
       'facilitytype', 'status', 'areas', 'other-name',
       ...
       'units-of-m', 'status-legend', 'count-of-semi', 'multi-country',
       'areas-subnat-sat-display', 'capacity-details', 'wiki-from-name',
       'tracker-display', 'tracker-legend', 'pci-list'],
      dtype='object', length=162)
Index(['id', 'url', 'name', 'unit-name', 'status', 'areas', 'other-name',
       'fuel', 'owner', 'parent', 'start-year', 'capacity', 'location',
       'prefecture/district', 'state/province', 'region', 'geometry',
       'tracker', 'fuel-filter', 'maturity', 'tracker-custom', 'other-local',
       'subnat', 'subnat2', 'region2', 'pci5', 'pci6', 'pid', 'prod-gas',
       'prod-year-gas', 'tracker-acro', 'search', 'scaling-capacity',
       'capacity-table', 'units-of-m', 'status-legend',
       'areas-subnat-sat-display', 'tracker-display', 'pci-list'],
      dtype='object')
europe
Saving file for map europe
This is len 

In [ ]:
gdf = gpd.read_file('/content/drive/MyDrive/output_from_colab/europe_2025-02-06.geojson')

# gdf['start-year']
# gdf['fuel-filter']


# CHECK with PMs about the new filters to make sense TODO
hy_check = gdf[gdf['fuel-filter']=='methane']

for col in ['maturity', 'pci-list']:
    print(col)
    print(set(hy_check[col].to_list()))
    mat = hy_check[hy_check['maturity']!='none']
    print(mat[['maturity', 'tracker-custom', 'fuel-filter']])

issues_df = pd.concat([mat])

# print(set(issues_df['tracker-custom'].to_list()))
# print(len(pci_b))
# print(len(pci_5))
# print(len(mat))


issues_df.to_csv('/content/drive/MyDrive/output_from_colab/issues_df_hy_sections.csv')

maturity
{'none'}
Empty DataFrame
Columns: [maturity, tracker-custom, fuel-filter]
Index: []
pci-list
{'none', 'both', '5'}
Empty DataFrame
Columns: [maturity, tracker-custom, fuel-filter]
Index: []


In [ ]:
for name, gdf in renamed_one_gdf_by_map.items():

    print(gdf[gdf['name']=='Montoir LNG Terminal'])
    print(gdf['geometry'])


     terminalid projectid       id                                        url  \
4437      T0456         1  T045601  https://www.gem.wiki/Montoir_LNG_Terminal   
4438      T0456         0  T045600  https://www.gem.wiki/Montoir_LNG_Terminal   

                      name  unit_name facilitytype     status   areas  \
4437  Montoir LNG Terminal  Expansion       Import   Proposed  France   
4438  Montoir LNG Terminal                  Import  Operating  France   

     other-name  ... mou-for-h2-supply? contract-for-h2-supply?  \
4437             ...                NaN                     NaN   
4438             ...                NaN                     NaN   

     financing-for-supply-of-h2?  \
4437                         NaN   
4438                         NaN   

     co-located-with-electrolyzer/h2-production-facility?  \
4437                                                NaN     
4438                                                NaN     

     what-%-of-h2-blending-currently?  \


In [ ]:
# create list in javascript format of all countries listed in country_to_check in the entire gdf sorted alphabetically
def create_new_country_list(final_dict_gdfs):
  all_countries = []

  for mapname, gdf in final_dict_gdfs.items():
      full = set(gdf['areas'].to_list())
      # add all countries together and then find unique
      print(len(full))
      print(full)
      for item in full:
        # split it all out

        items = item.split('; ')

        for lil_item in items:
            lil_item = lil_item.strip('; ')
            if lil_item in all_countries:
                pass
            else:
                all_countries.append(lil_item)

  return sorted(all_countries)


all_countries = create_new_country_list(final_dict_gdfs)
print(all_countries)


187
{'Serbia;', 'North Macedonia; Albania;', 'Belgium; United Kingdom;', 'Bulgaria; Greece;', 'Slovakia; Hungary;', 'Algeria;', 'Norway; France;', 'Denmark;', 'Belarus;', 'Russia; Belarus; Ukraine;', 'Bosnia and Herzegovina; Croatia;', 'Russia; Estonia; Latvia;', 'Poland; Slovakia;', 'Türkiye; Greece; Serbia; Hungary; Austria;', 'Croatia; Slovenia;', 'Moldova;', 'Greece; Italy;', 'Cyprus;', 'Ukraine; Kazakhstan; Russia;', 'Moldova; Ukraine;', 'Italy; Slovenia; Hungary;', 'Finland; Estonia; Latvia; Lithuania; Poland; Germany;', 'United Kingdom; Germany; Norway;', 'Russia; Germany;', 'Belarus; Poland; Slovakia;', 'Algeria; Spain;', 'Romania; Bulgaria;', 'Switzerland; Belgium;', 'Estonia;', 'Austria; Czech Republic;', 'Hungary; Slovenia; Italy;', 'Greece; Albania; Italy;', 'Israel; Cyprus; Greece;', 'Qatar; Saudi Arabia; Jordan; Syria; Türkiye; Bulgaria;', 'Georgia; Romania;', 'Algeria; Italy;', 'Finland;', 'North Macedonia; Bulgaria;', 'Libya; Italy;', 'Montenegro;', 'Belgium; France; Lu